### requirements

+ beautifulsoup4 (4.10.0)
+ lxml (4.8.0)
+ wikipedia (1.4.0)
+ wikipedia-api (0.5.4)
+ nltk (3.6.5)
+ nltk.download("punkt")

### input ndb data (XML TEI encoded)

+ 27 volumes split into "_head" and "_leben" 
+ "_leben" contains life sections for biographies of a particular volume
+ "_head" contains head and post life sections for biographies of a particular volume

In [1]:
# format cells using black
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import os
import re
import bs4
import requests
import pandas as pd

import wikipedia
import wikipediaapi

import nltk

# from nltk.tokenize import sent_tokenize
# nltk.download("punkt")

from nltk.tokenize import sent_tokenize
from time import sleep
from typing import List

<IPython.core.display.Javascript object>

In [3]:
class Error(Exception):
    """Base class for other exceptions."""

    pass


class KeyNDBNError(Error):
    """Attribute <n> is missing in the parsed biography."""

    pass


class KeyNDBSFZError(Error):
    """Attribute <ndb:sfz> is missing in the parsed biography."""

    pass


class KeyNDBSEXError(Error):
    """Attribute <ndb:sex> is missing in the parsed biography."""

    pass


class BadStatusCodeError(Error):
    """Raised when status code of request does not equal 200."""

    pass


class Biography:
    def __init__(
        self,
        ndb_data: bs4.element.Tag,
        ndb_text: str,
        ndb_n: str,
        ndb_sfz: str,
        ndb_sex: str,
    ):
        self.ndb_data = ndb_data  # xml tei encoded
        self.ndb_text = ndb_text  # text (extracted)
        self.ndb_n = ndb_n  # volume-entry-version ?
        self.ndb_sfz = ndb_sfz  # (unique) ndb biography id
        self.ndb_sex = ndb_sex  # 1:= male, 2:= female, 9:=unknown
        self.gnd = None  # (person) id - resolved via sfz-gnd mapping
        self.ndb_name = None  # extracted from _head file by sfz
        self.wikipedia_title = None  # resolved by gnd mapping
        self.wikipedia_text = None  # extracted using wikiapi

    def __repr__(self):
        return f"{self.wikipedia_title=} {self.ndb_n=} {self.gnd=} {self.ndb_sfz=}"

<IPython.core.display.Javascript object>

In [4]:
path = os.path.abspath("")
data_ndb_volumes_path_rel = "data/ndb_volumes/"
data_ndb_volumes_path_abs = os.path.join(path, data_ndb_volumes_path_rel)

pattern_ndb_volume_id = re.compile(r"^bsb_.*_(\d{2})_leben.xml$")
ndb_volume_leben_sorted = []

for filename in os.listdir(data_ndb_volumes_path_abs):
    if filename.endswith(".xml") and "leben" in filename:
        ndb_volume_id = pattern_ndb_volume_id.search(filename).group(1)
        ndb_volume_leben_sorted.append((ndb_volume_id, filename))
ndb_volume_leben_sorted = [x[1] for x in sorted(ndb_volume_leben_sorted)]

print(ndb_volume_leben_sorted)

['bsb_00016233_01_leben.xml', 'bsb_00016318_02_leben.xml', 'bsb_00016319_03_leben.xml', 'bsb_00016320_04_leben.xml', 'bsb_00016321_05_leben.xml', 'bsb_00016322_06_leben.xml', 'bsb_00016325_07_leben.xml', 'bsb_00016409_08_leben.xml', 'bsb_00016326_09_leben.xml', 'bsb_00016327_10_leben.xml', 'bsb_00016328_11_leben.xml', 'bsb_00016329_12_leben.xml', 'bsb_00016330_13_leben.xml', 'bsb_00016332_14_leben.xml', 'bsb_00016333_15_leben.xml', 'bsb_00016334_16_leben.xml', 'bsb_00016335_17_leben.xml', 'bsb_00016336_18_leben.xml', 'bsb_00016337_19_leben.xml', 'bsb_00016338_20_leben.xml', 'bsb_00016339_21_leben.xml', 'bsb_00016410_22_leben.xml', 'bsb_00019558_23_leben.xml', 'bsb_00020000_24_leben.xml', 'bsb_00020001_25_leben.xml', 'bsb_00020002_26_leben.xml', 'bsb_20000003_27_leben.xml']


<IPython.core.display.Javascript object>

### parse _leben

In [5]:
biographies_dict = {}
error_count = {
    "KeyNDBNError": 0,
    "KeyNDBSFZError": 0,
    "KeyNDBSEXError": 0,
    "AssertionError": 0,
}

for i, filename in enumerate(ndb_volume_leben_sorted):
    print(f"processing volume: {i+1} / 27", end="\r")
    with open(os.path.join(data_ndb_volumes_path_abs, filename), "r") as file:
        soup = bs4.BeautifulSoup(file, "lxml")
        biographies = soup.find_all("div", {"type": "leben"})

        for biography in biographies:
            try:
                if not biography.has_attr("n"):
                    raise KeyNDBNError
                n = biography["n"]
                if not biography.has_attr("ndb:sfz"):
                    raise KeyNDBSFZError
                sfz = biography["ndb:sfz"]
                if not biography.has_attr("ndb:sex"):
                    raise KeyNDBSEXError
                sex = biography["ndb:sex"]
                assert isinstance(biography, bs4.element.Tag)
                assert isinstance(n, str)
                assert isinstance(sfz, str)
                assert isinstance(sex, str)
                text = " ".join([x.text for x in biography.find_all("p")])
                text = text.strip().replace("\n", " ")  # TODO check "" or " "
                biography_obj = Biography(
                    ndb_data=biography, ndb_text=text, ndb_n=n, ndb_sfz=sfz, ndb_sex=sex
                )
                biographies_dict[sfz] = biography_obj
            except KeyNDBNError:
                error_count["KeyNDBNError"] += 1
            except KeyNDBSFZError:
                error_count["KeyNDBSFZError"] += 1
            except KeyNDBSEXError:
                error_count["KeyNDBSEXError"] += 1
            except AssertionError:
                error_count["AssertionError"] += 1

print(
    "\n",
    error_count,
    "\n",
    f"{sum(error_count.values())} biographies can't be parsed due to invalid xml schmeme.",
)

processing volume: 27 / 27
 {'KeyNDBNError': 1, 'KeyNDBSFZError': 0, 'KeyNDBSEXError': 1, 'AssertionError': 0} 
 2 biographies can't be parsed due to invalid xml schmeme.


<IPython.core.display.Javascript object>

In [6]:
path = os.path.abspath("")
data_ndb_volumes_path_rel = "data/ndb_volumes/"
data_ndb_volumes_path_abs = os.path.join(path, data_ndb_volumes_path_rel)

pattern_ndb_volume_id = re.compile(r"^bsb_.*_(\d{2})_head.xml$")
ndb_volume_head_sorted = []

for filename in os.listdir(data_ndb_volumes_path_abs):
    if filename.endswith(".xml") and "head" in filename:
        ndb_volume_id = pattern_ndb_volume_id.search(filename).group(1)
        ndb_volume_head_sorted.append((ndb_volume_id, filename))
ndb_volume_head_sorted = [x[1] for x in sorted(ndb_volume_head_sorted)]

print(ndb_volume_head_sorted)

['bsb_00016233_01_head.xml', 'bsb_00016318_02_head.xml', 'bsb_00016319_03_head.xml', 'bsb_00016320_04_head.xml', 'bsb_00016321_05_head.xml', 'bsb_00016322_06_head.xml', 'bsb_00016325_07_head.xml', 'bsb_00016409_08_head.xml', 'bsb_00016326_09_head.xml', 'bsb_00016327_10_head.xml', 'bsb_00016328_11_head.xml', 'bsb_00016329_12_head.xml', 'bsb_00016330_13_head.xml', 'bsb_00016332_14_head.xml', 'bsb_00016333_15_head.xml', 'bsb_00016334_16_head.xml', 'bsb_00016335_17_head.xml', 'bsb_00016336_18_head.xml', 'bsb_00016337_19_head.xml', 'bsb_00016338_20_head.xml', 'bsb_00016339_21_head.xml', 'bsb_00016410_22_head.xml', 'bsb_00019558_23_head.xml', 'bsb_00020000_24_head.xml', 'bsb_00020001_25_head.xml', 'bsb_00020002_26_head.xml', 'bsb_20000003_27_head.xml']


<IPython.core.display.Javascript object>

### parse _head

In [7]:
pattern_ndb_volume_id = re.compile(r"^bsb_.*_(\d{2})_head.xml$")
error_count_head = {"KeyNDBSFZError": 0, "AssertionError": 0}

for i, filename in enumerate(ndb_volume_head_sorted):
    ndb_volume_id = int(pattern_ndb_volume_id.search(filename).group(1))
    print(f"processing volume: {i+1} / 27 | {filename}", end="\r")
    with open(os.path.join(data_ndb_volumes_path_abs, filename), "r") as file:
        soup = bs4.BeautifulSoup(file, "lxml")
        biographies = soup.find_all("div", {"type": "entry"})
        biographies = biographies + soup.find_all("div", {"type": "entry_fam"})

        for biography in biographies:
            try:
                if not biography.has_attr("ndb:sfz"):
                    raise KeyNDBSFZError
                sfz = biography["ndb:sfz"]
                assert isinstance(sfz, str)

                head = biography.find("div", {"type": "kopf"})
                p = head.find("p")
                pers_name = p.find("persname")
                # TODO refactor name! mult names/persname...
                name_list = pers_name.find_all("hi")
                name_list = [x.text.replace("\n", " ") for x in name_list]
                # name_first = name_list[-1]
                # name_last = name_list[0]
                name = " ".join(name_list)

                if sfz in biographies_dict:
                    biographies_dict[sfz].ndb_name = name
            except KeyNDBSFZError:
                # skips <wrong> tags - not a problem
                error_count_head["KeyNDBSFZError"] += 1
            except AssertionError:
                error_count_head["AssertionError"] += 1
print("\n", error_count_head)

processing volume: 27 / 27 | bsb_20000003_27_head.xml
 {'KeyNDBSFZError': 1369, 'AssertionError': 0}


<IPython.core.display.Javascript object>

In [8]:
missing_ndb_name_in_header = 0
for k, v in biographies_dict.items():
    if v.ndb_name is None:
        missing_ndb_name_in_header += 1
print(f"{missing_ndb_name_in_header} (ndb-)names can't be parsed from the _head files.")
# not really necessary, because corresponding wikipedia article must exist - that title can be used

6 (ndb-)names can't be parsed from the _head files.


<IPython.core.display.Javascript object>

### (example) Einstein, Albert

+ sfz: sfz68290
+ https://www.deutsche-biographie.de/sfz68290.html#ndbcontent
+ http://data.deutsche-biographie.de/rest/sfz68290.xml

In [9]:
# missing gnd
biographies_dict["sfz68290"]

self.wikipedia_title=None self.ndb_n='n04-404-03' self.gnd=None self.ndb_sfz='sfz68290'

<IPython.core.display.Javascript object>

In [10]:
biographies_dict["sfz68290"].ndb_text

'Selten wohl hat ein Mensch das „Gesetz, danach er angetreten“ so zwingend in sich gefühlt und ist ihm, zunächst in kindlicher Unbewußtheit, allen äußeren Einflüssen zum Trotz gefolgt, wie E.Einstein. Als gereifter Gelehrter erinnerte er sich noch einiger Gedanken aus seiner Kindheit, die deutlich den Zug zu dem verraten, was wir heute Relativitätstheorie nennen. Von allem Wissensstoff, den das Gymnasium vermittelte, berührte ihn näher einzig und allein die euklidischeGeometrie; noch im Alter pries er sie begeistert als eine notwendige Vorstufe für jede Beschäftigung mit theoretischer Physik, weil sie die innere Konsequenz, die Geschlossenheit besitzt, welche für die Physik das anzustrebende, wenn auch nie völlig zu erreichende Ziel ist. Zu dieser Veranlagung kam eine ebenso früh scharf ausgeprägte und mit den damaligen Zuständen keineswegs konforme politische Einstellung, die sich wohl zum Teil aus seiner jüdischen Abkunft erklärt. Zum „guten Schüler“ eignete er sich unter diesen Umst

<IPython.core.display.Javascript object>

## sfz-gnd mapping

### static mapping (data/gnd/beacon_ndb_extended.txt)

In [11]:
gnd_dict = {}
sfz_dict = {}

path = os.path.abspath("")
data_gnd_path_rel = "data/gnd/"
data_gnd_path_abs = os.path.join(path, data_gnd_path_rel)
pattern_gnd = re.compile(r"^[\d\w]+")
pattern_sfz = re.compile(r"sfz[\d]+$")

with open(data_gnd_path_abs + "beacon_ndb_extended.txt", "r") as f:
    for line in f:
        gnd_match = pattern_gnd.search(line)
        sfz_match = pattern_sfz.search(line)

        if gnd_match and sfz_match:
            gnd_id = gnd_match.group(0)
            sfz_id = sfz_match.group(0)

            if gnd_id not in gnd_dict:
                gnd_dict[gnd_id] = sfz_id
            if sfz_id not in sfz_dict:
                sfz_dict[sfz_id] = gnd_id

<IPython.core.display.Javascript object>

In [12]:
# (example) get gnd for sfz: sfz68290
print(sfz_dict["sfz68290"])

118529579


<IPython.core.display.Javascript object>

In [13]:
gnd_mapping_exists = 0
missing_gnd_for_sfz = 0

for sfz in biographies_dict:
    if sfz in sfz_dict:
        gnd_mapping_exists += 1
    else:
        missing_gnd_for_sfz += 1

sum_missing_and_not_missing = gnd_mapping_exists + missing_gnd_for_sfz
print(
    f"{gnd_mapping_exists} / {sum_missing_and_not_missing} ({round(gnd_mapping_exists/sum_missing_and_not_missing,4)}%) sfz id's can be mapped to their gnd (via this static solution) - {missing_gnd_for_sfz} gnd id's are missing."
)

23462 / 24182 (0.9702%) sfz id's can be mapped to their gnd (via this static solution) - 720 gnd id's are missing.


<IPython.core.display.Javascript object>

In [14]:
missing_gnd_sfz = []
for sfz in biographies_dict:
    if sfz not in sfz_dict:
        missing_gnd_sfz.append(sfz)

<IPython.core.display.Javascript object>

### http request (for missing gnd mapping)
+ http://data.deutsche-biographie.de/beta/solr-open/?q=id:{sfzid}&fl=id,defgnd&wt=json
+ example (sfz68290): http://data.deutsche-biographie.de/beta/solr-open/?q=id:sfz68290&fl=id,defgnd&wt=json

In [15]:
url_prefix = "http://data.deutsche-biographie.de/beta/solr-open/?q=id:"
url_suffix = "&fl=id,defgnd&wt=json"
missing_gnd_in_response_sfz_id = []
processed_sfz_success = []

for i, sfz in enumerate(missing_gnd_sfz):
    print(f"requesting gnd: {i+1} / {len(missing_gnd_sfz)}", end="\r")
    try:
        response = requests.get(url=url_prefix + sfz + url_suffix, timeout=10)
        status_code = response.status_code
        if status_code != 200:
            raise BadStatusCodeError(f"Bad request, status code: {status_code}.")
        data = response.json()
        gnd = data["response"]["docs"][0]["defgnd"]
        if sfz not in sfz_dict:
            sfz_dict[sfz] = gnd
        if gnd not in gnd_dict:
            gnd_dict[gnd] = sfz
        processed_sfz_success.append(sfz)
        sleep(0.2)
    except (requests.exceptions.Timeout, BadStatusCodeError, KeyError, IndexError):
        missing_gnd_in_response_sfz_id.append(sfz)
    except requests.ConnectionError as e:
        # Max retries exceeded
        missing_gnd_in_response_sfz_id.append(sfz)
    except ValueError as e:
        # includes JSONDecodeError
        missing_gnd_in_response_sfz_id.append(sfz)

<IPython.core.display.Javascript object>

In [16]:
print(len(processed_sfz_success), len(missing_gnd_in_response_sfz_id))
print(
    f"{len(missing_gnd_in_response_sfz_id)} biographies have to be droped from the corpus due to missing gnd.",
)

655 65
65 biographies have to be droped from the corpus due to missing gnd.


<IPython.core.display.Javascript object>

In [17]:
# Add gnd from sfz_mapping_dict to biograpies_obj_dict
for sfz, biography in biographies_dict.items():
    if sfz in sfz_dict:
        biography.gnd = sfz_dict[biography.ndb_sfz]

<IPython.core.display.Javascript object>

In [18]:
biographies_dict_sfz = biographies_dict
biographies_dict_gnd = {}

number_missing_gnd = 0
number_multible_sfz_for_gnd = 0

for sfz, biography in biographies_dict_sfz.items():
    gnd = biography.gnd
    if gnd:
        if (
            gnd not in biographies_dict_gnd
        ):  # ~5 occurances with multible sfz mapping to same gnd (?)
            biographies_dict_gnd[gnd] = biography
        else:
            number_multible_sfz_for_gnd += 1
    else:
        number_missing_gnd += 1
print(f"{number_missing_gnd=}, {number_multible_sfz_for_gnd=}")

number_missing_gnd=65, number_multible_sfz_for_gnd=5


<IPython.core.display.Javascript object>

### gnd wikipedia mapping (static)

+ help
https://de.wikipedia.org/wiki/Hilfe:GND
+ wikipedia articles linked to gnd (daily updates)
https://persondata.toolforge.org/beacon/dewiki_list.txt

In [19]:
path = os.path.abspath("")
data_gnd_wiki_path_rel = "data/gnd/"
data_gnd_wiki_path_abs = os.path.join(path, data_gnd_wiki_path_rel)

with open(data_gnd_wiki_path_abs + "beacon_dewiki_list.txt", "r") as f:
    for line in f:
        title, gnd, timestamp = line.split("|")
        if gnd in biographies_dict_gnd:
            biographies_dict_gnd[gnd].wikipedia_title = title

<IPython.core.display.Javascript object>

In [20]:
gnd_wiki_mapping_failure = []
for k, v in biographies_dict_gnd.items():
    if not v.wikipedia_title:
        gnd_wiki_mapping_failure.append(k)

<IPython.core.display.Javascript object>

In [22]:
volume_dict = {}
for k, v in biographies_dict_gnd.items():
    if v.gnd in gnd_wiki_mapping_failure:
        volume = v.ndb_n[1:3]
        if volume in volume_dict:
            volume_dict[volume] += 1
        else:
            volume_dict[volume] = 1
assert len(gnd_wiki_mapping_failure) == sum(volume_dict.values())

<IPython.core.display.Javascript object>

In [23]:
volume_dict

{'01': 233,
 '02': 218,
 '03': 206,
 '04': 159,
 '06': 145,
 '05': 163,
 '07': 169,
 '08': 142,
 '09': 168,
 '10': 171,
 '11': 134,
 '12': 166,
 '13': 132,
 '14': 127,
 '15': 115,
 '16': 102,
 '17': 99,
 '18': 123,
 '19': 111,
 '20': 107,
 '21': 105,
 '22': 91,
 '23': 108,
 '24': 82,
 '25': 98,
 '26': 96,
 '27': 57}

<IPython.core.display.Javascript object>

## wikipedia api
+ https://pypi.org/project/Wikipedia-API/
+ https://wikipedia.readthedocs.io/en/latest/code.html

+ max 200 requests per second (sleep for 5ms after every request)

In [24]:
# this process takes ~4h
wiki = wikipediaapi.Wikipedia("de")
missing_wikipedia_title_for_biography_count = 0
wikipedia_title_doesnt_exist_count = 0
connection_error_msgs = []
i = 0
l = len(biographies_dict_gnd)
for k, v in biographies_dict_gnd.items():
    i += 1
    print(
        f"processing volume: {i} / {l} | {missing_wikipedia_title_for_biography_count} {wikipedia_title_doesnt_exist_count} {len(connection_error_msgs)}",
        end="\r",
    )
    if v.wikipedia_title:  # not None
        try:
            wikipedia_page = wiki.page(v.wikipedia_title)
            if wikipedia_page.exists():
                wikipedia_text = wikipedia_page.text.replace("\n", " ")
                v.wikipedia_text = wikipedia_text
            else:
                wikipedia_title_doesnt_exist_count += 1
        except Exception as e:
            connection_error_msgs.append(str(e))
        sleep(0.05)
    else:
        missing_wikipedia_title_for_biography_count += 1

<IPython.core.display.Javascript object>

In [26]:
# check connection_error_msgs for exception type
if connection_error_msgs:
    print(connection_error_msgs[0])

<IPython.core.display.Javascript object>

In [27]:
missing_wikipedia_data = 0
missing_gnd = 0
print(f"biographies pre pop: {len(biographies_dict_gnd)}")

for k in list(biographies_dict_gnd):
    v = biographies_dict_gnd[k]
    pop = False
    if not (v.wikipedia_title and v.wikipedia_text):
        missing_wikipedia_data += 1
        pop = True
    if not v.gnd:
        missing_gnd += 1
        pop = True
    if pop:
        if k in biographies_dict_gnd:
            biographies_dict_gnd.pop(k)
        if v.ndb_sfz in biographies_dict_sfz:
            biographies_dict_sfz.pop(v.ndb_sfz)


print(f"{missing_gnd=}, {missing_wikipedia_data=}")
print(f"biographies post pop: {len(biographies_dict_gnd)}")

biographies pre pop: 24112
missing_gnd=0, missing_wikipedia_data=3628
biographies post pop: 20484


<IPython.core.display.Javascript object>

In [28]:
data = {
    "ndb_n": [],
    "ndb_sfz": [],
    "gnd": [],
    "ndb_sex": [],
    "ndb_name": [],
    "wikipedia_title": [],
    "ndb_text": [],
    "wikipedia_text": [],
}
for k, v in biographies_dict_gnd.items():
    data["ndb_n"].append(v.ndb_n)
    data["wikipedia_title"].append(v.wikipedia_title)
    data["ndb_sfz"].append(v.ndb_sfz)
    data["gnd"].append(v.gnd)
    data["ndb_sex"].append(v.ndb_sex)
    data["ndb_name"].append(v.ndb_name)
    data["ndb_text"].append(v.ndb_text)
    data["wikipedia_text"].append(v.wikipedia_text)
assert len(data["ndb_n"]) == len(data["wikipedia_title"])
assert len(data["wikipedia_title"]) == len(data["ndb_sfz"])
assert len(data["ndb_sfz"]) == len(data["gnd"])
assert len(data["gnd"]) == len(data["ndb_sex"])
assert len(data["ndb_sex"]) == len(data["ndb_name"])
assert len(data["ndb_name"]) == len(data["ndb_text"])
assert len(data["ndb_text"]) == len(data["wikipedia_text"])

<IPython.core.display.Javascript object>

In [29]:
df_ndb = pd.DataFrame.from_dict(data, dtype="string")
df_ndb

,ndb_n,ndb_sfz,gnd,ndb_sex,ndb_name,wikipedia_title,ndb_text,wikipedia_text
0,n01-001-01,sfz45545,118643525,1,"Aachen Hans von (Johann von Achen, Hans Ach)",Hans von Aachen,"Karel van Mander (Schilderboeck, Haarlem 1604 ...",Hans von Aachen (* 1552 in Köln; † 4. März 161...
1,n01-001-02,sfz15,118500015,1,"Aal (Anguilla), Johannes",Johannes Aal,A.Aal verfocht zur Zeit der Reformation die al...,Johannes Aal (* um 1500 in Bremgarten AG; † 28...
2,n01-002-01,sfz17,104198273,1,dall' Abaco Evaristo Felice,Evaristo Felice Dall’Abaco,A.Abaco kam aus Verona über Modena 1704 an den...,Evaristo Felice Dall’Abaco (* 12. Juli 1675 in...
3,n01-002-02,sfz19,100002307,1,Abbadie Jacques,Jacques Abbadie,A.Abbadie studierte auf den Akademien von Saum...,"Jacques Abbadie, auch James, Jacobus oder Jaco..."
4,n01-002-03,sfz7153,118646419,1,Abbe Ernst Carl,Ernst Abbe,"Die Einsicht der Lehrer, die die außerordentli...",Ernst Karl Abbe [ˈabə] (* 23. Januar 1840 in ...
...,...,...,...,...,...,...,...,...
20479,n27-908-01,sfz141086,1116026791,2,Westphal-Hellbusch Sigrid Hellbusch,Sigrid Westphal-Hellbusch,Nach der Reifeprüfung am I. Oberlyzeum in Berl...,Sigrid Westphal-Hellbusch (* 10. Juni 1915 in ...
20480,n27-909-01,sfz60626,117327328,1,Westphalen Ferdinand,Ferdinand von Westphalen,W. Westphalen besuchte das Gymnasium in Salzw...,Ferdinand Otto Wilhelm Henning von Westphalen ...
20481,n27-910-01,sfz85259,117327573,1,Westrumb Friedrich,Johann Friedrich Westrumb,W. Westrumb begann 1764 eine Lehre in der kgl...,Johann Friedrich Westrumb (* 2. Dezember 1751 ...
20482,n27-911-01,sfz141099,1051891450,1,Wetter Ernst,Ernst Wetter,Nach der Volksschule absolvierte W. Wetter d...,Ernst Wetter (* 27. August 1877 in Töss (heute...


<IPython.core.display.Javascript object>

### save dataframe

In [30]:
# assume data/df/ exists
path = os.path.abspath("")
data_df_rel = "data/df/"
data_df_ndb_abs = os.path.join(path, data_df_rel, "df_ndb_wikipedia.pkl")
df_ndb.to_pickle(data_df_ndb_abs)

<IPython.core.display.Javascript object>

### load dataframe

In [31]:
# format cells using black
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [32]:
path = os.path.abspath("")
data_df_rel = "data/df/"
data_df_ndb_abs = os.path.join(path, data_df_rel, "df_ndb_wikipedia.pkl")
corpus = pd.read_pickle(data_df_ndb_abs)
corpus

,ndb_n,ndb_sfz,gnd,ndb_sex,ndb_name,wikipedia_title,ndb_text,wikipedia_text
0,n01-001-01,sfz45545,118643525,1,"Aachen Hans von (Johann von Achen, Hans Ach)",Hans von Aachen,"Karel van Mander (Schilderboeck, Haarlem 1604 ...",Hans von Aachen (* 1552 in Köln; † 4. März 161...
1,n01-001-02,sfz15,118500015,1,"Aal (Anguilla), Johannes",Johannes Aal,A.Aal verfocht zur Zeit der Reformation die al...,Johannes Aal (* um 1500 in Bremgarten AG; † 28...
2,n01-002-01,sfz17,104198273,1,dall' Abaco Evaristo Felice,Evaristo Felice Dall’Abaco,A.Abaco kam aus Verona über Modena 1704 an den...,Evaristo Felice Dall’Abaco (* 12. Juli 1675 in...
3,n01-002-02,sfz19,100002307,1,Abbadie Jacques,Jacques Abbadie,A.Abbadie studierte auf den Akademien von Saum...,"Jacques Abbadie, auch James, Jacobus oder Jaco..."
4,n01-002-03,sfz7153,118646419,1,Abbe Ernst Carl,Ernst Abbe,"Die Einsicht der Lehrer, die die außerordentli...",Ernst Karl Abbe [ˈabə] (* 23. Januar 1840 in ...
...,...,...,...,...,...,...,...,...
20479,n27-908-01,sfz141086,1116026791,2,Westphal-Hellbusch Sigrid Hellbusch,Sigrid Westphal-Hellbusch,Nach der Reifeprüfung am I. Oberlyzeum in Berl...,Sigrid Westphal-Hellbusch (* 10. Juni 1915 in ...
20480,n27-909-01,sfz60626,117327328,1,Westphalen Ferdinand,Ferdinand von Westphalen,W. Westphalen besuchte das Gymnasium in Salzw...,Ferdinand Otto Wilhelm Henning von Westphalen ...
20481,n27-910-01,sfz85259,117327573,1,Westrumb Friedrich,Johann Friedrich Westrumb,W. Westrumb begann 1764 eine Lehre in der kgl...,Johann Friedrich Westrumb (* 2. Dezember 1751 ...
20482,n27-911-01,sfz141099,1051891450,1,Wetter Ernst,Ernst Wetter,Nach der Volksschule absolvierte W. Wetter d...,Ernst Wetter (* 27. August 1877 in Töss (heute...


<IPython.core.display.Javascript object>

In [33]:
corpus.loc[corpus["ndb_sex"] == "1"].head(3)

,ndb_n,ndb_sfz,gnd,ndb_sex,ndb_name,wikipedia_title,ndb_text,wikipedia_text
0,n01-001-01,sfz45545,118643525,1,"Aachen Hans von (Johann von Achen, Hans Ach)",Hans von Aachen,"Karel van Mander (Schilderboeck, Haarlem 1604 ...",Hans von Aachen (* 1552 in Köln; † 4. März 161...
1,n01-001-02,sfz15,118500015,1,"Aal (Anguilla), Johannes",Johannes Aal,A.Aal verfocht zur Zeit der Reformation die al...,Johannes Aal (* um 1500 in Bremgarten AG; † 28...
2,n01-002-01,sfz17,104198273,1,dall' Abaco Evaristo Felice,Evaristo Felice Dall’Abaco,A.Abaco kam aus Verona über Modena 1704 an den...,Evaristo Felice Dall’Abaco (* 12. Juli 1675 in...


<IPython.core.display.Javascript object>

In [34]:
corpus.loc[corpus["ndb_sex"] == "2"].head(3)

,ndb_n,ndb_sfz,gnd,ndb_sex,ndb_name,wikipedia_title,ndb_text,wikipedia_text
66,n01-033-01,sfz111,118688669,2,Achler Elisabeth Maria die gute Beth,Elisabeth Achler,Unter dem Einfluß des Augustiner-Chorherrn Kon...,Elisabeth Achler oder Elsbeth Achler (auch „El...
70,n01-035-02,sfz122,118646753,2,Ackermann Charlotte Maria Magdalena,Charlotte Ackermann,A.Ackermann war der gefeierte Liebling des Ham...,Marie Magdalene Charlotte Ackermann (* 23. Aug...
71,n01-035-03,sfz123,11600570X,2,Ackermann Dorothea Caroline,Dorothea Ackermann,A.Ackermann gehörte zu den glänzendsten Darste...,Caroline Dorothea Elisabeth Ackermann (* 12. F...


<IPython.core.display.Javascript object>

In [35]:
corpus.loc[corpus["ndb_sex"] == "9"].head(3)

,ndb_n,ndb_sfz,gnd,ndb_sex,ndb_name,wikipedia_title,ndb_text,wikipedia_text
17,n01-010-01,sfz36,138044252,9,Abel,Brüder Abel,Florian († 14.5.1565 Prag?) war lange in Prag ...,"Die Brüder Abel, Bernhard Abel († 1563) und Ar..."
28,n01-015-02,sfz61,122318331,9,Abenberg Grafen von,Abenberg (Adelsgeschlecht),Die Familie führte den erblichen Titel wahrsch...,Die Familie von Abenberg war ein altes fränkis...
31,n01-017-01,sfz6,123455316,9,Abensberg.,Abensberg (Adelsgeschlecht),"Abensberg, Hauptname eines seit 1065 mit Eberh...",Grafen von Abensberg (auch Abensberger) waren ...


<IPython.core.display.Javascript object>

In [36]:
# get ndb text by gnd
gather_ndb_text_by_gnd_example = corpus.loc[corpus["gnd"] == "118860445"][
    "ndb_text"
].values[0]
# rheinisch-westfälisches Dynastengeschlecht.
# https://www.deutsche-biographie.de/sfz1194.html#ndbcontent
gather_ndb_text_by_gnd_example

'Der im Jahre 1166 als erster Träger dieses Namens begegnende edelfreie Heinrich von Arberg ist der Sohn des Burggrafen Gerhard von Köln, wobei nicht feststellbar ist, ob Gerhard in verwandtschaftlicher Beziehung zu den seit 1032 genannten Burggrafen steht; wegen des bei seinen Nachkommen vorkommenden Vornamens liegt eine solche Beziehung bei Gerhards unmittelbarem Vorgänger, dem 1136-59 erwähnten Burggrafen Heinrich, nahe. Der nachweisbare Besitz der Edelherren von A.Arenberg liegt in der Eifel an der oberen Ahr, wo vermutlich Gerhards Sohn Heinrich auf der Kuppe des A.Arenbergs die Burg erbaute, nach der seine Nachkommen sich benannten, ferner im Erfttal (Helpenstein), in Morenhoven (Kreis Bonn-Land), ander Sieg und im Westerwald (Schönstein, Wildenberg, Rosbach und Altenwied). Von dem Geschlecht zweigte sich höchstwahrscheinlich mit Gerhard von Wildenberg das Edelherrengeschlecht von Wildenberg (Sitz Wildenberg, Kreis Altenkirchen/Rheinland) in der ersten Hälfte des 13. Jahrhunderts

<IPython.core.display.Javascript object>

In [37]:
# get wikipedia text by gnd
gather_wikipedia_text_by_gnd_example = corpus.loc[corpus["gnd"] == "118860445"][
    "wikipedia_text"
].values[0]
# http://tools.wmflabs.org/persondata/redirect/gnd/de/118860445
gather_wikipedia_text_by_gnd_example

'Das Haus Arenberg (auch Aremberg) ist ein Adelsgeschlecht des deutschen Hochadels, das in der Eifel ansässig war, nach der Burg Aremberg im Landkreis Ahrweiler benannt wurde und nach dem Aussterben der ursprünglichen Familie von Arenberg im Mannesstamm eine Seitenlinie der Grafen von der Mark bzw. später der Herren von Ligne darstellt. Der ursprüngliche Herrschaftsbereich war die Herrschaft, später Grafschaft beziehungsweise Herzogtum Arenberg. Dieses Gebiet ging im Zuge des Ersten Koalitionskrieges unter. Nach dem Reichsdeputationshauptschluss entstand das Herzogtum Arenberg-Meppen. Der gegenwärtige Chef des Hauses Arenberg führt den Titel Herzog, wohingegen die verbliebenen Familienmitglieder die Titel Prinz bzw. Prinzessin innehaben.  Geschichte 1. Edelfreie von Arenberg, Burggrafen von Köln, 12. Jahrhundert Die edelfreie Familie von Arenberg ist 1117–1129 erschließbar und 1166 erstmals erwähnt. Sie hatte zeitweise das Amt des Burggrafen in Köln inne, das sie 1279 an den Erzbischof

<IPython.core.display.Javascript object>

In [38]:
# gnd:=104198273
ndb_sample = corpus.loc[corpus["gnd"] == "104198273"]["ndb_text"].values[0]
ndb_sample

'A.Abaco kam aus Verona über Modena 1704 an den kurbayerischen Hof nach München und folgte als Hofcellist dem Kurfürsten Max Emanuel ins Brüsseler Exil. 1715 kehrte er als Konzertmeister nach München zurück, wurde 1717 „Churfürstlicher Rath“ und trat 1740 in den Ruhestand. – A.Abacos Instrumentalmusik zählt zur reifen spätbarocken italienischen Kunst: In ihrem Gleichgewicht von pathetischer Haltung und formaler Vollkommenheit, Ausdruckskraft und Klanggestaltung weist sie klassische Züge auf.'

<IPython.core.display.Javascript object>

In [39]:
wikipedia_sample = corpus.loc[corpus["gnd"] == "104198273"]["wikipedia_text"].values[0]
# http://tools.wmflabs.org/persondata/redirect/gnd/de/104198273
wikipedia_sample

"Evaristo Felice Dall’Abaco (* 12. Juli 1675 in Verona; † 12. Juli 1742 in München) war ein italienischer Violinist, Cellist und Komponist.  Leben Evaristo Felice Dall’Abacos Vater war der causidico Damiano Abaco, seine Mutter Clorinda Abaco. Ein causidico war ein „Rechtsgelehrter niederen Grades ohne die Befugnisse, Prozesschriften einzureichen und bei Rechtsprechungen zu plädieren.“Dall’Abaco war als Violinist und Cellist möglicherweise ein Schüler Giuseppe Torellis. 1696 trat er in Modena des Öfteren mit Tomaso Antonio Vitali auf. Abacos musikalische Tätigkeiten in Modena wurden durch von ihm quittierte Zahlungsbelege im Staatsarchiv dokumentiert. Er war kein ständiges Mitglied der Hofkapelle, wirkte aber bei besonderen Anlässen mit. Dies waren Opernaufführungen; Kirchenmusiken, Akademien und Hoffeste. Er lernte die Opern und Oratorien der spätvenetianischen Schule kennen, wie Werke von Gianettini, Pallavicini, Lotti und Marc' Antonio Ziani. Er kam aber auch schon früh mit dem franz

<IPython.core.display.Javascript object>

## todo check volume 27 head/leben... different structure...?

+ sfz regex only digits? no? check ndb help
+ ndb name wrong ... multiple personname

In [40]:
def sentence_too_short(sent: str) -> bool:
    # only evaluate sentences with more or equal to 5 words (token)
    tokens = sent.split(" ")
    if len(tokens) < 5:
        return True
    return False


def contains_equal_sentence(ndb: str, wikipedia: str) -> bool:
    ndb_sent = sent_tokenize(ndb, language="german")
    wikipedia_sent = sent_tokenize(wikipedia, language="german")
    for sent in ndb_sent:
        if sentence_too_short(sent):
            continue
        if sent in wikipedia_sent:
            return True
    return False


def get_equal_sentences(ndb: str, wikipedia: str) -> List[str]:
    equal_sentences = []
    ndb_sent = sent_tokenize(ndb, language="german")
    wikipedia_sent = sent_tokenize(wikipedia, language="german")
    for sent in ndb_sent:
        if sentence_too_short(sent):
            continue
        if sent in wikipedia_sent:
            equal_sentences.append(sent)
    return equal_sentences

<IPython.core.display.Javascript object>

In [41]:
equal_sentence_occurance_gnd = {}
number_rows = corpus[corpus.columns[0]].count()

for index, row in corpus.iterrows():
    print(f"processing row: {index} / {number_rows}", end="\r")
    ndb_text = row["ndb_text"]
    wikipedia_text = row["wikipedia_text"]
    gnd = row["gnd"]
    if contains_equal_sentence(ndb_text, wikipedia_text):
        if gnd not in equal_sentence_occurance_gnd:
            equal_sentence_occurance_gnd[gnd] = get_equal_sentences(
                ndb_text, wikipedia_text
            )

<IPython.core.display.Javascript object>

In [42]:
print(len(equal_sentence_occurance_gnd))

249


<IPython.core.display.Javascript object>

In [43]:
equal_sentence_occurance_gnd["128428430"]

['Seitdem verliert sich seine Spur.']

<IPython.core.display.Javascript object>

In [44]:
corpus.loc[corpus["gnd"] == "128428430"]["ndb_text"].values[0]

'A.Adriani, ein getaufter spanischer Jude, taucht zum ersten Mal in Italien in einer „Introductio ad hebraicam linguam“ auf, die 1501 und 1508 bei Aldus Manutius in Venedig erschien, dann in der griechischen Grammatik des Konstantinos Laskaris (ebenda 1512). Erasmus von Rotterdam rühmt seinen guten Ruf als Hebraist in Italien und Deutschland. Ursprünglich war A.Adriani Arzt (Dr. med.doctor medicinae). 1512 war er in Tübingen; hier erschien 1513 bei Thomas Anshelm sein „Libellus horam faciendi pro Domino“ mit Übersetzungen christlicher Gebete ins Hebräische. Im gleichen Jahr kam er mit Empfehlungsbriefen von Johann Reuchlin (Tübingen) und Konrad Pellican (Pforzheim) über Straßburg nach Basel, in der Absicht, über Venedig ins Heilige Land zu pilgern. Er unterrichtete in Basel Wolfgang Fabricius Capito, die Söhne des Buchdruckers Johannes Amerbach, in Heidelberg Johann Brenz und Johannes Oecolampadius; 1517 erhielt er auf Empfehlung des Erasmus den hebraistischen Lehrstuhl am „Collegium T

<IPython.core.display.Javascript object>

In [45]:
corpus.loc[corpus["gnd"] == "128428430"]["wikipedia_text"].values[0]

'Matthäus Adriani (* um 1475 in Spanien; † nach 1521 in Freiburg im Breisgau?) war ein jüdischer Hebraist.  Leben Adriani wurde in Spanien als Kind einer jüdischen Familie geboren. Erstmals tritt er 1501 in Italien in Erscheinung, als er eine Instruktion der hebräischen Sprache in Venedig verfasst. Zu diesem Zeitpunkt ist er bereits Doktor der Medizin. Bis 1512 verbleibt er in Venedig, wo er die griechische Grammatik des Aldus Manutius bearbeitet und Erasmus von Rotterdam auf ihn aufmerksam wird und seine Arbeit hervorhebt. Dadurch erhält er auch einen guten Ruf in Deutschland, so dass er 1512 in Tübingen erscheint. Hier verfasste er 1513 Übersetzungen christlicher Gebete ins Hebräische. Auf Empfehlung von Johannes Reuchlin und Konrad Pellican (1478–1556) kam er im selben Jahr über Straßburg nach Basel, in der Absicht, über Venedig ins Heilige Land zu pilgern. Er unterrichtete in Basel Wolfgang Capito, die Söhne des Buchdruckers Johann Amerbach, Johannes Brenz und Johannes Oekolampadiu

<IPython.core.display.Javascript object>